# Headfixed odor exposure

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import h5py as h5
import os
import time
import itertools
import scipy.stats
import custom

In [ ]:
source_file = './data/headfixed.h5'
processed_file = './data/headfixed_cleaned.h5'

features = ['pupil', 'track']

# Parameters
pre = -12000
post = 22000
bin_size = 200
xticklabels = np.arange(-10000, 22000, 5000)
xticks = (xticklabels - pre) / bin_size

frame_dur = 200
epoch_ctrl = 'h2o'

idx = pd.IndexSlice

In [ ]:
cmap = custom.diverging_cmap(np.array([60, 179, 113]) / 255., np.array([24, 116, 205]) / 255.)
cmap2 = custom.diverging_cmap(np.array([100, 100, 100]) / 255., np.array([186, 141, 56]) / 255.)

# Import and setup data

In [ ]:
def pupil_trials(df, axis=1, level=-1, trial_index='trials', bin_size=200, pre=-16000, post=21000):
    ''' Organize data into trials
    Trials are defined by column `trial_index`.
    '''

    pre_bins = pre / bin_size
    post_bins = post / bin_size
    column_names = df.columns.names

    # Find unique animals from certain pattern in experiment names
    # subjects = np.unique([x.split('_')[0] for x in df.columns.levels[0]])

    exps = df.xs(trial_index, axis=axis, level=level).columns
    exps_dict = {}

    # Iterate over experiments (different planes are separate experiments)
    for exp in exps:
        epochs_dict = {}

        # Iterate over epochs
        for epoch in df[exp].index.levels[0]:
            df_epoch = df.loc[epoch, exp]
            trial_starts = df_epoch[df_epoch[trial_index] == 1].index
            trial_window = zip(trial_starts + pre, trial_starts + post - bin_size)
            trials_dict = {}      # Initialize list of dataframes for each trial

            # Iterate over trials
            for n, x in enumerate(trial_window):
                # Get subset of dataframe for current trial window
                trial = df_epoch.loc[slice(*x), :]

                # Define new index (based on time relative to trial)
                # Need to define index relative to current index in case of missing data.
                index_vals = np.arange(pre, post, bin_size)
                time_trial_time = pd.DataFrame({
                    'trial_time': index_vals,
                    'time': np.linspace(*(x + (len(index_vals), )), dtype=int)
                })
                time_trial_time = time_trial_time.set_index('time')

                # Add and set new index
                trial = pd.concat([trial, time_trial_time], axis=1)
                trial.columns.names = [column_names[-1]]
                trial = trial.reset_index(level='time')
                trial = trial.set_index('trial_time')
                trials_dict[n] = trial

            trials_df = pd.concat(trials_dict, names=['trial'], axis=1)
            epochs_dict[epoch] = trials_df
            
        epochs_df = pd.concat(epochs_dict, names=['epoch'], axis=0)
        exps_dict[exp] = epochs_df
        
    exps_df = pd.concat(exps_dict, names=exps.names, axis=axis)
    exps_df = exps_df.sort_index(axis=axis)  # Allows for indexing (killed myself over this)

    return exps_df


def get_trials(df, axis=1, level=-1, trial_index='trials', bin_size=200, pre=-16000, post=21000):
    ''' Organize data into trials
    Trials are defined by index `trial_index` in axis `axis` and level `level`. 
    '''

    pre_bins = pre / bin_size
    post_bins = post / bin_size
    column_names = df.columns.names

    # Find unique animals from certain pattern in experiment names
    # subjects = np.unique([x.split('_')[0] for x in df.columns.levels[0]])

    exps = df.xs(trial_index, axis=axis, level=level).columns
    exps_dict = {}

    # Iterate over experiments (different planes are separate experiments)
    for exp in exps:
        epochs_dict = {}

        # Iterate over epochs
        for epoch in df[exp].index.levels[0]:
            df_epoch = df.loc[epoch, exp]
            trial_starts = df_epoch[df_epoch[trial_index] == 1].index
            trial_window = zip(trial_starts + pre, trial_starts + post - bin_size)
            trials_dict = {}      # Initialize list of dataframes for each trial

            # Iterate over trials
            for n, x in enumerate(trial_window):
                # Get subset of dataframe for current trial window
                trial = df_epoch.loc[slice(*x), :]

                # Define new index (based on time relative to trial)
                # Need to define index relative to current index in case of missing data.
                index_vals = np.arange(pre, post, bin_size)
                trial_index = pd.DataFrame({
                    'trial_time': index_vals,
                    'time': np.linspace(*(x + (len(index_vals), )), dtype=int)
                })
                trial_index = trial_index.set_index('time')

                # Add and set new index
                trial = pd.concat([trial, trial_index], axis=1)
                trial.columns.names = [column_names[-1]]
                trial = trial.reset_index(level='time')
                trial = trial.set_index('trial_time')
                trials_dict[n] = trial

            trials_df = pd.concat(trials_dict, names=['trial'], axis=1)
            epochs_dict[epoch] = trials_df
            
        epochs_df = pd.concat(epochs_dict, names=['epoch'], axis=0)
        exps_dict[exp] = epochs_df
        
    exps_df = pd.concat(exps_dict, names=exps.names, axis=axis)
    exps_df = exps_df.sort_index(axis=axis)  # Allows for indexing (killed myself over this)

    return exps_df

## Import data

In [ ]:
with pd.HDFStore(filename, mode='r') as hf:
    # Only get first plane and remove baseline epoch
    df_behav = hf['behav'].loc[(['h2o', 'odor']), (slice(None), slice(None), 'P1')]
    df_neural = hf['neural'][neural_type].loc[(['h2o', 'odor']), (slice(None), slice(None), 'P1')].astype(float)  # need to cast since it's object type
    
    # Clean
    df_behav.columns = df_behav.columns.droplevel(level='plane')
    df_neural.columns = df_neural.columns.droplevel(level='plane')

## Normalize data

Data is **scaled** so that one unit is equal to standard deviation (calculated during water exposure for a particular neuron).  
Data is **centered** on its median activity during water exposure.

In [ ]:
# Standardize variance (std = 1) to baseline
stds = df_neural.loc[epoch_ctrl].std()
df_neural_norm = (df_neural - df_neural.loc[epoch_ctrl].median()) / stds

Behavior data is standardized (divide by standard deviation). Track is NOT centered on its mean. Absolute data is taken on track to get 'movement' measure.

In [ ]:
# Clean & normalize

df_behav_orig = df_behav.copy()

# Normalize pupil
ix_col = idx[:, :, :, 'pupil']
filtered = df_behav.loc[:, ix_col].groupby(level='epoch').rolling(5, center=True).median()
filtered.index = filtered.index.droplevel(0)
df_behav.loc[:, ix_col] = filtered
pupil_mean = filtered.loc[epoch_ctrl, ix_col].mean()
pupil_std = filtered.loc[epoch_ctrl, ix_col].std()
df_behav.loc[:, ix_col] -= pupil_mean
df_behav.loc[:, ix_col] *= 1 / pupil_std

# Normalize track
# Take absolute value to capture 'movement'
ix_col = idx[:, :, :, 'track']
filtered = df_behav.loc[:, ix_col].groupby(level='epoch').rolling(5, center=True).mean()
filtered.index = filtered.index.droplevel(0)
df_behav.loc[:, ix_col] = np.abs(filtered)

## Create trial structure

In [ ]:
df = pd.concat([df_behav, df_neural_norm], axis=1)
df = df.sort_index(axis=1)
df_trials = pupil_trials(df, bin_size=bin_size, pre=pre, post=post)

# Break up data into neural and behavioral
neural = df_trials.iloc[:, [not isinstance(s, basestring) for s in df_trials.columns.get_level_values(-1)]]
neural.columns.names = df_trials.columns.names[:-1] + ['neuron']
# behav = df_trials.iloc[:, [isinstance(s, basestring) for s in df_trials.columns.get_level_values(-1)]]
behav = df_trials.loc[:, idx[:, :, :, :, ['pupil', 'track']]]
behav.columns.names = df_trials.columns.names[:-1] + ['feature']

# Add epoch names
neural.loc[:, 'time_epoch'] = [
    '0 pre' if -12000 <= ts < -6000 else
    '1 na' if -6000 <= ts < 0 else
    '2 response' if 0 <= ts < 10000 else
    '3 na' if 10000 <= ts < 16000 else
    '4 post' if 16000 <= ts < 22000 else
    'na'
    for ts in neural.reset_index('trial_time')['trial_time']
]
neural = neural.set_index('time_epoch', append=True).reorder_levels(['epoch', 'time_epoch', 'trial_time'])

behav.loc[:, 'time_epoch'] = [
    '0 pre' if -12000 <= ts < -6000 else
    '1 na' if -6000 <= ts < 0 else
    '2 response' if 0 <= ts < 10000 else
    '3 na' if 10000 <= ts < 16000 else
    '4 post' if 16000 <= ts < 22000 else
    'na'
    for ts in behav.reset_index('trial_time')['trial_time']
]
behav = behav.set_index('time_epoch', append=True).reorder_levels(['epoch', 'time_epoch', 'trial_time'])

In [ ]:
with pd.HDFStore(output_data) as df_out:
    df_out['neural'] = neural#_response
    df_out['behav'] = behav#_response

# Import processed data

In [ ]:
with pd.HDFStore(processed_file) as df_out:
    neural_response = df_out['neural']
    behav_response = df_out['behav']

# Keep first two trials and first exposure (order A) only
neural_response = neural_response.loc[:, idx['TMT', :, 'A', [0, 1]]]
behav_response = behav_response.loc[:, idx['TMT', :, 'A', [0, 1]]]

# Create long format dataframe
neural_response_long = neural_response.stack(list(range(neural_response.columns.nlevels))).reset_index().rename({0: 'value'}, axis=1)
behav_response_long = behav_response.stack(list(range(behav_response.columns.nlevels))).reset_index().rename({0: 'value'}, axis=1)

# Visualize data

## Behavior

In [ ]:
# Pupil and velocity plots

def behavior_plots(data, plt_name=None, vmin=-2, vmax=2):
    fig, axes = plt.subplots(2, 2, sharex=True, sharey='row', figsize=(12, 5))
    fig_cb, ax_cb = plt.subplots(figsize=(0.25, 4))
    
    for n, (epoch, df) in enumerate(data.groupby(axis=0, level='epoch')):
        df.index = df.index.droplevel('epoch')
        
        C = df.copy()
        C.index = C.index.droplevel('time_epoch')
        sns.heatmap(C.T, ax=axes[0, n], cbar_ax=ax_cb, center=0, cmap=cmap2, vmin=vmin, vmax=vmax)
        im = axes[0, n].collections[0]
        
        Y = df.mean(axis=1)
        E = df.sem(axis=1)
        Y.plot(ax=axes[1, n])
        X = axes[1, n].lines[0].get_xdata()
        axes[1, n].fill_between(X, Y + E, Y - E, alpha=0.2)
        axes[0, n].set_xticks(xticks)
        axes[0, n].set_xticklabels(xticklabels)

for feature, v in zip(['pupil', 'track'], [[-5, 5], [0, 250]]):
    ix_col = idx[:, :, :, :, feature]
    behavior_plots(behav_response.loc[:, ix_col], vmin=v[0], vmax=v[1])

In [ ]:
data = behav_response_long.groupby(['epoch', 'time_epoch', 'subject', 'feature']).mean()['value'].reset_index()

time_epochs = ['0 pre', '2 response', '4 post']
g = sns.FacetGrid(data, col='feature', hue='epoch', hue_order=['h2o', 'odor'], sharey=False)
g.map(sns.pointplot, 'time_epoch', 'value', order=time_epochs, ci=68).add_legend()

## Neural data

In [ ]:
# Neural plots

def activity_heatmap(data, max_z=4):
    fig, axes  = plt.subplots(ncols=2, sharey='row', sharex=True, figsize=(8, 4))
    fig_cb, ax_cb = plt.subplots(figsize=(0.25, 4))

    ims = []
    for n, (grp, df) in enumerate(data.groupby(axis=0, level='epoch')):
        df.index = df.index.droplevel('epoch')
        
        # Plot heatmap
        C = df.copy()
        C.columns = np.arange(C.shape[1])
        C.index = C.index.droplevel('time_epoch')
        sns.heatmap(C.T, ax=axes[n], cbar_ax=ax_cb, cmap=cmap, vmin=-max_z, vmax=max_z, yticklabels=C.shape[1] - 1)
        axes[n].set_title(grp)
        ims.append(axes[n].collections[0])
        
        # Remove spines
        sns.despine(ax=axes[n], left=True, bottom=True)
        sns.despine(ax=axes[n], left=True, bottom=True)

    axes[0].set_ylabel('Neurons');
    axes[0].set_ylabel('Average neural response');
    
    fig.tight_layout()

# Sorting
sort_ix = np.argsort(neural_response.loc[('odor', '2 response'), :].mean().mean(level=['subject', 'neuron']).as_matrix())[::-1]

# Plot
data = neural_response.mean(axis=1, level=['subject', 'neuron']).iloc[:, sort_ix]
activity_heatmap(data)

In [ ]:
data = neural_response_long.groupby(['experiment', 'epoch', 'time_epoch', 'order', 'subject', 'neuron']).mean()['value'].reset_index()

time_epochs = ['0 pre', '2 response', '4 post']
sns.pointplot(data=data, x='time_epoch', y='value', hue='epoch', order=time_epochs, ci=68)

# Group neurons

## Statistical difference

_Should activity during exposure be subtracted by average before conveyor movement??_

In [ ]:
thresh_data = neural_response.mean(axis=1, level=['experiment', 'subject', 'neuron'])
n_neurons = thresh_data.shape[1]
alpha = 0.05
p_thresh = alpha / n_neurons  # Bonferroni correction

grouping = pd.DataFrame(
    columns=pd.MultiIndex.from_product([['h2o', 'odor'], ['p', 'sign', 'group']]),
    index=thresh_data.columns,
    dtype=float
)

for ep in ['h2o', 'odor']:
    for n, neuron in enumerate(thresh_data):
        X = thresh_data.loc[(ep, '0 pre'), neuron]
        Y = thresh_data.loc[(ep, '2 response'), neuron]
        if len(np.unique(np.concatenate((X, Y)))) > 1:
            _, grouping.loc[neuron, (ep, 'p')] = scipy.stats.mannwhitneyu(X, Y)
            grouping.loc[neuron, (ep, 'sign')] = np.sign(Y.mean() - X.mean())

        if grouping.loc[neuron, (ep, 'p')] < p_thresh:
            grouping.loc[neuron, (ep, 'group')] = grouping.loc[neuron, (ep, 'sign')] 
        else:
            grouping.loc[neuron, (ep, 'group')] = 0

grouping.loc[:, idx[:, ['sign', 'group']]] = grouping.loc[:, idx[:, ['sign', 'group']]].astype(int)

In [ ]:
X = grouping[('odor', 'group')].value_counts().sort_index()

fig, ax = plt.subplots()
ax.pie(X, labels=X.index, autopct='%1.1f%%')
circle = plt.Circle((0, 0), 0.7071, color='white')
ax.add_artist(circle)
plt.axis('equal');

In [ ]:
data = pd.concat([thresh_data.T, grouping.loc['group'].T], axis=1).set_index(['h2o', 'odor'], append=True).T
data.index = pd.MultiIndex.from_tuples(data.index, names=['epoch', 'time_epoch', 'trial_time'])

fig, axes = plt.subplots(nrows=2, ncols=3, sharey=True, figsize=(15, 10))

for ax, (grp, df) in zip(axes.T, data.groupby(axis=0, level='epoch')):
    Y = data.loc[grp].mean(axis=1, level=['odor']).rolling(5, center=True).mean()
    E = data.loc[grp].sem(axis=1, level=['odor']).rolling(5, center=True).mean()
    Y.plot(ax=ax[0]);
    X = ax[0].lines[0].get_xdata()
    
    for n in range(Y.shape[1]):
        y = Y.as_matrix()[:, n]
        e = E.as_matrix()[:, n]
        ix = ~ (np.isnan(y) + np.isnan(e))
        ax[0].fill_between(X[ix], y[ix] - e[ix], y[ix] + e[ix], alpha=0.25)
    
#     Y1 = data.loc[(grp, ['0 pre', '2 response']), :].mean(axis=0, level='time_epoch').mean(axis=1, level=['subject', 'odor'])
#     y = Y1.mean(axis=1, level=['odor'])
#     e = Y1.sem(axis=1, level=['odor'])
#     y.plot(ax=ax[1], yerr=e, marker='o')

# Correlation

In [ ]:
features = ['pupil', 'track']

behav_corr_data = behav_response.loc[(slice(None), ['0 pre', '2 response', '4 post']), :].unstack('epoch').stack('trial')
neural_corr_data = neural_response.loc[(slice(None), ['0 pre', '2 response', '4 post']), :].unstack('epoch').stack('trial')

behav_corr_data = behav_corr_data.groupby(level='time_epoch').apply(lambda x: x.reset_index(drop=True))
neural_corr_data = neural_corr_data.groupby(level='time_epoch').apply(lambda x: x.reset_index(drop=True))

corrs = np.zeros((4, neural_corr_data.shape[1]), dtype=float) * np.nan
corrs = pd.DataFrame(corrs, index=pd.MultiIndex.from_product([features, ['r', 'p']]), columns=neural_corr_data.columns)
for obs_key in neural_corr_data:
    exp, subj, _, neuron, epoch = obs_key
    obs_neural = neural_corr_data[obs_key]
    for feature in features:
        obs_behav = behav_corr_data[subj, feature, epoch]
        na_ix = obs_behav.isna() | obs_neural.isna()
        corrs.loc[feature, obs_key] = scipy.stats.pearsonr(obs_behav[~na_ix].dropna(), obs_neural[~na_ix].dropna())

In [ ]:
# Plot

b = pd.concat([corrs.stack('epoch'), grouping.loc['group']], axis=0).T.set_index(['h2o', 'odor'], append=True).T
b.index = pd.MultiIndex.from_tuples(b.index, names=['feature', 'stat', 'epoch'])

fig, axes = plt.subplots(ncols=2, sharex=True, sharey=True, figsize=(10, 5))
for n, feature in enumerate(features):
#     ctrl = corrs.loc[(feature, 'r'), (slice(None), slice(None), 'h2o')].dropna()
#     exp = corrs.loc[(feature, 'r'), (slice(None), slice(None), 'odor')].dropna()
    datasets = {}
    for ep in ['h2o', 'odor']:
        for grp in [-1, 0, 1]:
            datasets[(ep, grp)] = Y = b.loc[(feature, 'r', ep), (slice(None), slice(None), slice(None), grp)].dropna()
            custom.cdf(Y, ax=axes[n], label=' '.join([ep, str(grp)]), ignore_nan=False)
            if ep == 'h2o' and grp == -1:
                Y_ = Y
    axes[n].legend()
    axes[n].set_xlabel('r')

    pairs = itertools.combinations(datasets.keys(), 2)
    post_hocs = []
    for (x_ep, x_grp), (y_ep, y_grp) in pairs:
        xset = b.loc[(feature, 'r', x_ep), (slice(None), slice(None), slice(None), x_grp)]
        yset = b.loc[(feature, 'r', y_ep), (slice(None), slice(None), slice(None), y_grp)]
        w, p = scipy.stats.mannwhitneyu(xset, yset)
        post_hocs.append('{} vs {} p val: {}'.format((x_ep, x_grp), (y_ep, y_grp), p))
    axes[n].set_title('\n'.join(post_hocs))

In [ ]:
# Plot

b = pd.concat([corrs.stack('epoch'), grouping.loc['group']], axis=0).T.set_index(['h2o', 'odor'], append=True).T
b.index = pd.MultiIndex.from_tuples(b.index, names=['feature', 'stat', 'epoch'])

fig, axes = plt.subplots(ncols=2, sharex=True, sharey=True, figsize=(15, 5))
for n, feature in enumerate(features):
#     ctrl = corrs.loc[(feature, 'r'), (slice(None), slice(None), 'h2o')].dropna()
#     exp = corrs.loc[(feature, 'r'), (slice(None), slice(None), 'odor')].dropna()
    datasets = {}
    for ep in ['h2o', 'odor']:
        for grp in [-1, 0, 1]:
            datasets[(ep, grp)] = Y = b.loc[(feature, 'r', ep), (slice(None), slice(None), slice(None), grp)].dropna()
            custom.cdf(Y, ax=axes[n], label=' '.join([ep, str(grp)]), ignore_nan=False)
            if ep == 'h2o' and grp == -1:
                Y_ = Y
    axes[n].legend()
    axes[n].set_xlabel('r')

    pairs = itertools.combinations(datasets.keys(), 2)
    post_hocs = []
    for (x_ep, x_grp), (y_ep, y_grp) in pairs:
        xset = b.loc[(feature, 'r', x_ep), (slice(None), slice(None), slice(None), x_grp)]
        yset = b.loc[(feature, 'r', y_ep), (slice(None), slice(None), slice(None), y_grp)]
        u, p = scipy.stats.mannwhitneyu(xset, yset)
        post_hocs.append('{} vs {}, u: {}, p: {}'.format((x_ep, x_grp), (y_ep, y_grp), u, p))
    axes[n].set_title(feature + '\n' + '\n'.join(post_hocs))